In [2]:
from pyhvr import *
loadstd()

ModuleNotFoundError: No module named 'pyhvr'

In [ ]:
import umap.umap_ as umap#import umap
import textwrap
import plotly.express as px

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, BertForSequenceClassification, pipeline
from LeIA import SentimentIntensityAnalyzer

In [ ]:
from xgboost import XGBClassifier
from sklearn.dummy import DummyClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    auc,
    classification_report,
    confusion_matrix,
    f1_score,
)
from sklearn.preprocessing import LabelEncoder

## Load data

In [ ]:
data = pd.read_excel('dados/provas_tratadas_com_gabarito_e_area.xlsx',index_col=0)

In [ ]:
data = data[~((data.gabarito=='anulada')|(data.gabarito=='duplicada'))].copy()
data.reset_index(drop=True, inplace=True)

In [ ]:
full_question_text = [' '.join(x) for x in data[['caput'] + [f'alternativa_{i}' for i in [1,2,4,8,16]]].values]

In [ ]:
data['texto_completo_questao'] = full_question_text

In [ ]:
from cleantext import clean

In [ ]:
todas_questoes = ' '.join(full_question_text)
todas_questoes = clean(todas_questoes,no_digits=True,no_punct=True,no_numbers=True,replace_with_number='',replace_with_digit='')

In [ ]:
palavra,frequencia = np.unique(todas_questoes.split(),return_counts=True)
df_zipf = pd.DataFrame()
df_zipf['Palavra'] = palavra
df_zipf['Frequência'] = frequencia

In [ ]:
df_zipf.sort_values('Frequência',ascending=False).head(10)

In [ ]:
fre_ = df_zipf.sort_values('Frequência',ascending=False)['Frequência'].values
rank = np.arange(1,len(fre_)+1).astype(float)

In [ ]:
len(df_zipf)

In [ ]:
f, ax = plt.subplots(ncols=1,figsize=stdfigsize(nx=1))

logxyplot([[rank,fre_]],
          fmt='o',ax=ax,markeredgecolor='#202020',alpha=0.8,color='#bdbdbd')

logxyplot([[rank,7715*rank**-1]],ax=ax,linewidth=4,color='#3182bd',dashes=(5,2),label="$f\sim1/r$")


ax.set_xlabel('Ranque da palavra, $r$')
ax.set_ylabel('Frequência, $f$')
f.tight_layout()
#despite the diferences, the curves as quite similar.

In [ ]:
mapa_areas = {
"Artes": "Linguagens",
"Biologia": "Ciências da Natureza",
"Educação Física": "Linguagens",
"Espanhol": "Linguagens",
"Filosofia": "Ciências Humanas",
"Francês": "Linguagens",
"Física": "Ciências da Natureza",
"Geografia": "Ciências Humanas",
"História": "Ciências Humanas",
"Inglês": "Linguagens",
"Literatura": "Linguagens",
"Matemática": "Matemática",
"Português": "Linguagens",
"Química": "Ciências da Natureza",
"Sociologia": "Ciências Humanas",
}

In [ ]:
#import itertools
# mapa_soma_verdadeira = []
# for i in range(1,6):
#     for possibilidade in itertools.combinations([1,2,4,8,16],i):
#         mapa_soma_verdadeira+=[[np.sum(possibilidade),list(possibilidade)]]

In [ ]:
data['grande-area'] = data['area'].map(mapa_areas)

In [ ]:
mapa_soma_verdadeira = {
1: [1],
2: [2],
4: [4],
8: [8],
16: [16],
3: [1,2],
5: [1,4],
9: [1,8],
17: [1,16],
6: [2,4],
10: [2,8],
18: [2,16],
12: [4,8],
20: [4,16],
24: [8,16],
7: [1,2,4],
11: [1,2,8],
19: [1,2,16],
13: [1,4,8],
21: [1,4,16],
25: [1,8,16],
14: [2,4,8],
22: [2,4,16],
26: [2,8,16],
28: [4,8,16],
15: [1,2,4,8],
23: [1,2,4,16],
27: [1,2,8,16],
29: [1,4,8,16],
30: [2,4,8,16],
31: [1,2,4,8,16]
}

In [ ]:
data['alternativas_corretas'] = data.gabarito.map(mapa_soma_verdadeira)

In [ ]:
data['alternativas_falsas'] = data['alternativas_corretas'].apply(lambda x: sorted(list(set([1, 2, 4, 8, 16])-set(x))))

In [ ]:
data['texto_completo_questao_wrap'] = ["<br>".join(textwrap.wrap(texto,width=50)) for texto in data.texto_completo_questao]

In [ ]:
textos_das_alternativas_corretas = []
textos_das_alternativas_falsas = []

for i, dataum in data.iterrows():
    
    corretas = dataum.alternativas_corretas
    texto_das_alternativas_corretas = []
    if len(corretas)>0:
        for correta in corretas:
            texto_das_alternativas_corretas.append(dataum[f'alternativa_{correta}'])
    textos_das_alternativas_corretas.append(texto_das_alternativas_corretas)
    
    falsas = dataum.alternativas_falsas
    texto_das_alternativas_falsas = []
    if len(falsas)>0:
        for falsa in falsas:
            texto_das_alternativas_falsas.append(dataum[f'alternativa_{falsa}'])
    
    textos_das_alternativas_falsas.append(texto_das_alternativas_falsas)

In [ ]:
data['textos_das_alternativas_corretas'] = textos_das_alternativas_corretas
data['textos_das_alternativas_falsas'] = textos_das_alternativas_falsas

## Size distribution in number of chars

In [ ]:
data['tamanho_textos_das_alternativas_corretas'] = data.textos_das_alternativas_corretas.map(lambda x: [len(x_) for x_ in x])
data['tamanho_textos_das_alternativas_falsas'] = data.textos_das_alternativas_falsas.map(lambda x: [len(x_) for x_ in x])

In [ ]:
f, ax = plot([scipy_kde(np.concatenate(data.tamanho_textos_das_alternativas_falsas.values)).T],label='Falsas',color='#ef8a62',linewidth=3)
plot([scipy_kde(np.concatenate(data.tamanho_textos_das_alternativas_corretas.values)).T],label='Verdadeiras',ax=ax,color='#67a9cf',linewidth=3)

ax.set_title(fr" $\Delta=$ {np.mean(np.concatenate(data.tamanho_textos_das_alternativas_falsas.values))-np.mean(np.concatenate(data.tamanho_textos_das_alternativas_corretas.values)):.0f}")
ax.set_xlabel('Número de caracteres')
ax.set_ylabel('Distribuição de probabilidade');

In [ ]:
f, axs = plt.subplots(nrows=2,ncols=2,figsize=stdfigsize(nx=2,ny=2))
axs = axs.flatten()
for i,garea in enumerate(data['grande-area'].unique()):
    ax = axs[i]
    data_ = data[data['grande-area']==garea]
    plot([scipy_kde(np.concatenate(data_.tamanho_textos_das_alternativas_falsas.values)).T],label='Falsas',color='#ef8a62',linewidth=3,ax=ax)
    plot([scipy_kde(np.concatenate(data_.tamanho_textos_das_alternativas_corretas.values)).T],label='Verdadeiras',ax=ax,color='#67a9cf',linewidth=3)
    ax.set_title(garea+fr" $\Delta=$ {np.mean(np.concatenate(data_.tamanho_textos_das_alternativas_falsas.values))-np.mean(np.concatenate(data_.tamanho_textos_das_alternativas_corretas.values)):.0f}")
    ax.set_xlabel('Número de caracteres')
    ax.set_ylabel('Distribuição de probabilidade')
f.tight_layout()

In [ ]:
 len(data['area'].unique())

In [ ]:
f, axs = plt.subplots(nrows=5,ncols=3,figsize=stdfigsize(nx=3,ny=5))
axs = axs.flatten()
for i,garea in enumerate(data['area'].unique()):
    ax = axs[i]
    data_ = data[data['area']==garea]
    plot([scipy_kde(np.concatenate(data_.tamanho_textos_das_alternativas_falsas.values)).T],label='Falsas',color='#ef8a62',linewidth=3,ax=ax)
    plot([scipy_kde(np.concatenate(data_.tamanho_textos_das_alternativas_corretas.values)).T],label='Verdadeiras',ax=ax,color='#67a9cf',linewidth=3)
    ax.set_title(garea+fr" $\Delta=$ {np.mean(np.concatenate(data_.tamanho_textos_das_alternativas_falsas.values))-np.mean(np.concatenate(data_.tamanho_textos_das_alternativas_corretas.values)):.0f}")
    ax.set_xlabel('Número de caracteres')
    ax.set_ylabel('Distribuição de probabilidade')
f.tight_layout()

## Sentiment distribution (VADER)

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
def sentiment_text(lista):
    s = []
    for texto in lista:
        s.append(analyzer.polarity_scores(texto)['compound']*100)
    return s

In [ ]:
data['sentimento_das_alternativas_corretas'] = data['textos_das_alternativas_corretas'].apply(sentiment_text)
data['sentimento_das_alternativas_falsas'] = data['textos_das_alternativas_falsas'].apply(sentiment_text)

In [ ]:
f, ax = plot([scipy_kde(np.concatenate(data.sentimento_das_alternativas_falsas.values)).T],label='Falsas',color='#ef8a62',linewidth=3)
plot([scipy_kde(np.concatenate(data.sentimento_das_alternativas_corretas.values)).T],label='Verdadeiras',ax=ax,color='#67a9cf',linewidth=3)
ax.set_xlabel('Sentimento do texto (VADER)')
ax.set_ylabel('Distribuição de probabilidade')

In [ ]:
for garea in data['area'].unique():
    data_ = data[data['area']==garea]
    f, ax = plot([scipy_kde(np.concatenate(data_.sentimento_das_alternativas_falsas.values)).T],label='Falsas',color='#ef8a62',linewidth=3)
    plot([scipy_kde(np.concatenate(data_.sentimento_das_alternativas_corretas.values)).T],label='Verdadeiras',ax=ax,color='#67a9cf',linewidth=3)
    ax.set_title(garea+fr" $\Delta=$ {np.mean(np.concatenate(data_.sentimento_das_alternativas_falsas.values))-np.mean(np.concatenate(data_.sentimento_das_alternativas_corretas.values)):.0f}")
    ax.set_xlabel('Sentimento do texto (VADER)')
    ax.set_ylabel('Distribuição de probabilidade')

## Sentiment BERT

In [ ]:
finbert_pt_br_tokenizer = AutoTokenizer.from_pretrained("lucas-leme/FinBERT-PT-BR")
finbert_pt_br_model = BertForSequenceClassification.from_pretrained("lucas-leme/FinBERT-PT-BR")
finbert_pt_br_pipeline = pipeline(task='text-classification', model=finbert_pt_br_model, tokenizer=finbert_pt_br_tokenizer, device='cuda')

In [ ]:
def bert_sentiment(lista):
    s = []
    for texto in lista:
        res = finbert_pt_br_pipeline(texto)[0]
        if res['label']=='NEGATIVE':
            score = res['score']*-1
        elif res['label']=='POSITIVE':
            score = res['score']+1
        else:
            score = res['score']
        score = (score-(-1.))/(2.-(-1.))
        score = (score-0.5)*2*100
        s.append(score)
    return s

In [ ]:
data['bert_sentimento_das_alternativas_corretas'] = data['textos_das_alternativas_corretas'].apply(bert_sentiment)
data['bert_sentimento_das_alternativas_falsas'] = data['textos_das_alternativas_falsas'].apply(bert_sentiment)

In [ ]:
f, ax = plot([scipy_kde(np.concatenate(data.bert_sentimento_das_alternativas_falsas.values)).T],label='Falsas',color='#ef8a62',linewidth=3)
plot([scipy_kde(np.concatenate(data.bert_sentimento_das_alternativas_corretas.values)).T],label='Verdadeiras',ax=ax,color='#67a9cf',linewidth=3)
ax.set_xlabel('Sentimento do texto (BERT)')
ax.set_ylabel('Distribuição de probabilidade')

In [ ]:
for garea in data['area'].unique():
    data_ = data[data['area']==garea]
    f, ax = plot([scipy_kde(np.concatenate(data_.bert_sentimento_das_alternativas_falsas.values)).T],label='Falsas',color='#ef8a62',linewidth=3)
    plot([scipy_kde(np.concatenate(data_.bert_sentimento_das_alternativas_corretas.values)).T],label='Verdadeiras',ax=ax,color='#67a9cf',linewidth=3)
    ax.set_title(garea+fr" $\Delta=$ {np.mean(np.concatenate(data_.bert_sentimento_das_alternativas_falsas.values))-np.mean(np.concatenate(data_.bert_sentimento_das_alternativas_corretas.values)):.0f}")
    ax.set_xlabel('Sentimento do texto (BERT)')
    ax.set_ylabel('Distribuição de probabilidade')

## Embbeding

In [ ]:
# model = SentenceTransformer('neuralmind/bert-base-portuguese-cased')
model = SentenceTransformer('tgsc/sentence-transformer-ult5-pt-small')

In [ ]:
bert_ult5_pt_small_embbedding = data['texto_completo_questao'].apply(lambda x: model.encode(x))

In [ ]:
data['bert_ult5_pt_small_embbedding'] = bert_ult5_pt_small_embbedding

### Falsas vs. verdadeiras

In [ ]:
data['embbedding_textos_das_alternativas_corretas'] = data.textos_das_alternativas_corretas.map(lambda x: [model.encode(x_) for x_ in x])
data['embbedding_textos_das_alternativas_falsas'] = data.textos_das_alternativas_falsas.map(lambda x: [model.encode(x_) for x_ in x])

In [ ]:
embbedding_textos_das_alternativas_falsas = np.concatenate(data[((data.alternativas_falsas.apply(len)>0))]['embbedding_textos_das_alternativas_falsas'].values)
embbedding_textos_das_alternativas_corretas = np.concatenate(data[((data.alternativas_corretas.apply(len)>0))]['embbedding_textos_das_alternativas_corretas'].values)

In [ ]:
reducer = umap.UMAP(n_jobs=-1,n_neighbors=150,min_dist=0.93)

In [ ]:
umap_embbedding_textos_das_alternativas = reducer.fit_transform(np.vstack([embbedding_textos_das_alternativas_corretas,embbedding_textos_das_alternativas_falsas]))

In [ ]:
y = np.asarray([1]*len(embbedding_textos_das_alternativas_corretas)+[0]*len(embbedding_textos_das_alternativas_falsas))

In [ ]:
f, ax = plt.subplots(figsize=stdfigsize(scale=1.5))

mask_ = y==0

ax.scatter(
    umap_embbedding_textos_das_alternativas[mask_][:, 0],
    umap_embbedding_textos_das_alternativas[mask_][:, 1],
    s=25,
    marker='v',
    c='#ef8a62',
    label='Falsas'
)

mask_ = y==1

ax.scatter(
    umap_embbedding_textos_das_alternativas[mask_][:, 0],
    umap_embbedding_textos_das_alternativas[mask_][:, 1],
    s=25,
    marker='^',
    c='#67a9cf',
    label='Verdadeiras'
)
ax.legend()

# ax.set_xlim(0,15)
# ax.set_ylim(-2,8)

ax.set_xlabel('UMAP 1')
ax.set_ylabel('UMAP 2')

In [ ]:
X = np.concatenate([embbedding_textos_das_alternativas_falsas,embbedding_textos_das_alternativas_corretas])
y = np.asarray([1]*len(embbedding_textos_das_alternativas_corretas)+[0]*len(embbedding_textos_das_alternativas_falsas))

In [ ]:
#fração de alternativas corretas
np.mean(y)

In [ ]:
acc_xg = []
acc_dummy_mf = []
acc_dummy_st = []
acc_dummy_un = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y,random_state=i)
    
    clf_vf = XGBClassifier(subsample=0.6, eta=0.015)
    
    clf_vf_dummy_mf = DummyClassifier(strategy='most_frequent')
    clf_vf_dummy_st = DummyClassifier(strategy='stratified')
    clf_vf_dummy_un = DummyClassifier(strategy='uniform')
    
    clf_vf.fit(X_train,y_train);
    
    clf_vf_dummy_mf.fit(X_train,y_train);
    clf_vf_dummy_st.fit(X_train,y_train);
    clf_vf_dummy_un.fit(X_train,y_train);
    
    acc_xg_          = accuracy_score(y_train, clf_vf.predict(X_train)),accuracy_score(y_test, clf_vf.predict(X_test))
    acc_dummy_mf_    = accuracy_score(y_train, clf_vf_dummy_mf.predict(X_train)),accuracy_score(y_test, clf_vf_dummy_mf.predict(X_test))
    acc_dummy_st_ = accuracy_score(y_train, clf_vf_dummy_st.predict(X_train)),accuracy_score(y_test, clf_vf_dummy_st.predict(X_test))
    acc_dummy_un_ = accuracy_score(y_train, clf_vf_dummy_un.predict(X_train)),accuracy_score(y_test, clf_vf_dummy_un.predict(X_test))
    acc_xg.append(acc_xg_)
    acc_dummy_mf.append(acc_dummy_mf_)
    acc_dummy_st.append(clf_vf_dummy_st_)
    acc_dummy_un.append(clf_vf_dummy_un_)

In [ ]:
acc_xg = np.asarray(acc_xg)
acc_dummy_mf = np.asarray(acc_dummy_mf)
acc_dummy_st = np.asarray(acc_dummy_st)
acc_dummy_un = np.asarray(acc_dummy_un)

In [ ]:
avg_xgboost = np.asarray(
    [
        [np.mean(acc_dummy_un[::, 1]), np.std(acc_dummy_un[::, 1])],
        [np.mean(acc_dummy_st[::, 1]), np.std(acc_dummy_st[::, 1])],
        [np.mean(acc_dummy_mf[::, 1]), np.std(acc_dummy_mf[::, 1])],
        [np.mean(acc_xg[::, 1]), np.std(acc_xg[::, 1])],
    ]
)

In [ ]:
 avg_xgboost[::, 0]

In [ ]:
xgboost_labels = ["Uniforme","Estratificado","Mais frequente","xgboost"]

f, ax = plt.subplots(1, 1, figsize=stdfigsize(), sharex=True)
x_pos = list(range(len(xgboost_labels)))

ax.bar(x_pos, avg_xgboost[::, 0], yerr=1 * avg_xgboost[::, 1], color=["#9e9ac8","#74c476","#6baed6","#fc9272"])

ax.set_xticks(x_pos)
ax.set_xticklabels(xgboost_labels)
for x,avg_,sdt_ in zip(x_pos,avg_xgboost[::, 0],avg_xgboost[::, 1]):
    if x==3:
        ax.text(x,0.05,fr"{avg_:.3f}$\pm${sdt_:.3f}",ha='center',rotation=90,fontsize=22)
    else:
        ax.text(x,0.05,fr"{avg_:.3f}",ha='center',rotation=90,fontsize=22)
ax.tick_params(axis="x", rotation=45)
ax.set_ylabel("Acurácia")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y,random_state=np.argmax(acc_xg[::, 1]))

clf_vf = XGBClassifier(subsample=0.6, eta=0.015)
clf_vf.fit(X_train,y_train);
accuracy_score(y_test, clf_vf.predict(X_test))

In [ ]:
f, ax = plt.subplots(1, 1, figsize=stdfigsize(), sharex=True)

labels = ['Verdadeira','Falsa']

sns.heatmap(
    confusion_matrix(y_test, clf_vf.predict(X_test),normalize='true'),
    ax=ax,
    cmap="viridis_r",
    annot=True,
    annot_kws={"fontsize": 25},
    fmt=".2f",
    cbar=False,
)

ax.set_xticklabels(labels, size=20)
ax.set_yticklabels(labels, size=20)
ax.set_xlabel('Previsão do modelo')
ax.set_ylabel('Resposta correta')

### Áreas

In [ ]:
reducer = umap.UMAP(n_jobs=-1,n_neighbors=150,min_dist=0.3)
umap_bert_ult5_pt_small_embbedding = reducer.fit_transform(np.vstack(data.bert_ult5_pt_small_embbedding.values))
data['umap_bert_ult5_pt_small_embbedding_1'] = umap_bert_ult5_pt_small_embbedding.T[0]
data['umap_bert_ult5_pt_small_embbedding_2'] = umap_bert_ult5_pt_small_embbedding.T[1]

In [ ]:
areas = data['area'].unique()
colors = generate_color_list(np.asarray(list(range(len(areas)))))
markers = ['o', 'v', '^', '<', '>', 's', 'p', '*', 'h', 'H', '+', 'x', 'D', 'd', 'P', 'X', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [ ]:
f, ax = plt.subplots(figsize=stdfigsize(scale=2))

for i, area_ in enumerate(areas):
    ax.scatter(
        data[data['area'] == area_]['umap_bert_ult5_pt_small_embbedding_1'],
        data[data['area'] == area_]['umap_bert_ult5_pt_small_embbedding_2'],
        s=50,
        marker=markers[i],
        c=colors[i],
        label=area_
)

# ax.set_xlim(0,15)
# ax.set_ylim(-2,8)
ax.legend(ncols=1,loc='upper right',bbox_to_anchor=(1.2, 1.0))
ax.set_xlabel('UMAP 1')
ax.set_ylabel('UMAP 2')

In [ ]:
fig = px.scatter(data,
                 x='umap_bert_ult5_pt_small_embbedding_1',
                 y='umap_bert_ult5_pt_small_embbedding_2',
                 color='area',
                 custom_data=['texto_completo_questao_wrap'],
                 height=1000,
                 width=1200,
)

fig.update_traces(
    hovertemplate =
                "%{customdata[0]}",
    mode='markers',
    marker={'size': 10}
)

fig.show(config = {'scrollZoom': True,
                   })

In [ ]:
le = LabelEncoder()
y = le.fit_transform(data['area'])
X = np.vstack(data.bert_ult5_pt_small_embbedding.values)

In [ ]:
#fração de questões em cada área
[f"{area_}: {np.mean(data['area']==area_)}" for area_ in data['area'].unique()]

In [ ]:
f, ax = plt.subplots(ncols=2, figsize=stdfigsize(nx=2,ny=1.5))

df_ = pd.DataFrame([[area_, np.sum(data['area']==area_)] for area_ in data['area'].unique()],columns=['Disciplina','Número de questões'])
sns.barplot(x='Disciplina',y='Número de questões',data=df_.sort_values('Número de questões',ascending=False),hue='Disciplina',ax=ax[0])
ax[0].tick_params(axis="x", rotation=90)

df_ = pd.DataFrame([[area_, np.mean(data['area']==area_)] for area_ in data['area'].unique()],columns=['Disciplina','Fração de questões'])
sns.barplot(x='Disciplina',y='Fração de questões',data=df_.sort_values('Fração de questões',ascending=False),hue='Disciplina',ax=ax[1])
ax[1].tick_params(axis="x", rotation=90)
f.tight_layout()

In [ ]:
f, ax = plt.subplots(ncols=2, figsize=stdfigsize(nx=2,ny=1.5))

df_ = pd.DataFrame([[area_, np.sum(data['area']==area_)] for area_ in data['area'].unique()],columns=['Disciplina','Número de questões'])
sns.barplot(y='Disciplina',x='Número de questões',data=df_.sort_values('Número de questões',ascending=False),hue='Disciplina',ax=ax[0])
ax[0].tick_params(axis="x", rotation=0)
ax[0].set_xticks(np.arange(0, df_['Número de questões'].max() + 20, 20))

df_ = pd.DataFrame([[area_, np.mean(data['area']==area_)] for area_ in data['area'].unique()],columns=['Disciplina','Fração de questões'])
sns.barplot(y='Disciplina',x='Fração de questões',data=df_.sort_values('Fração de questões',ascending=False),hue='Disciplina',ax=ax[1])
ax[1].tick_params(axis="x", rotation=0)
ax[1].set_xlim(0, 0.12)  # Limita o eixo X até 0.12
ax[1].set_xticks(np.arange(0, 0.13, 0.02))  # Define os ticks com incrementos de 0.02
f.tight_layout()

In [ ]:
acc_xg = []
acc_dummy_mf = []
acc_dummy_st = []
acc_dummy_un = []
for i in tqdm.tqdm(range(10)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y,random_state=i)
    
    clf_vf = XGBClassifier()
    
    clf_vf_dummy_mf = DummyClassifier(strategy='most_frequent')
    clf_vf_dummy_st = DummyClassifier(strategy='stratified')
    clf_vf_dummy_un = DummyClassifier(strategy='uniform')
    
    clf_vf.fit(X_train,y_train);
    
    clf_vf_dummy_mf.fit(X_train,y_train);
    clf_vf_dummy_st.fit(X_train,y_train);
    clf_vf_dummy_un.fit(X_train,y_train);
    
    acc_xg_          = accuracy_score(y_train, clf_vf.predict(X_train)),accuracy_score(y_test, clf_vf.predict(X_test))
    acc_dummy_mf_    = accuracy_score(y_train, clf_vf_dummy_mf.predict(X_train)),accuracy_score(y_test, clf_vf_dummy_mf.predict(X_test))
    acc_dummy_st_ = accuracy_score(y_train, clf_vf_dummy_st.predict(X_train)),accuracy_score(y_test, clf_vf_dummy_st.predict(X_test))
    acc_dummy_un_ = accuracy_score(y_train, clf_vf_dummy_un.predict(X_train)),accuracy_score(y_test, clf_vf_dummy_un.predict(X_test))
    acc_xg.append(acc_xg_)
    acc_dummy_mf.append(acc_dummy_mf_)
    acc_dummy_st.append(clf_vf_dummy_st_)
    acc_dummy_un.append(clf_vf_dummy_un_)

In [ ]:
acc_xg = np.asarray(acc_xg)
acc_dummy_mf = np.asarray(acc_dummy_mf)
acc_dummy_st = np.asarray(acc_dummy_st)
acc_dummy_un = np.asarray(acc_dummy_un)

In [ ]:
avg_xgboost = np.asarray(
    [
        [np.mean(acc_dummy_un[::, 1]), np.std(acc_dummy_un[::, 1])],
        [np.mean(acc_dummy_st[::, 1]), np.std(acc_dummy_st[::, 1])],
        [np.mean(acc_dummy_mf[::, 1]), np.std(acc_dummy_mf[::, 1])],
        [np.mean(acc_xg[::, 1]), np.std(acc_xg[::, 1])],
    ]
)

In [ ]:
avg_xgboost[::, 0]

In [ ]:
xgboost_labels = ["Uniforme","Estratificado","Mais frequente","xgboost"]

f, ax = plt.subplots(1, 1, figsize=stdfigsize(), sharex=True)
x_pos = list(range(len(xgboost_labels)))

ax.bar(x_pos, avg_xgboost[::, 0], yerr=1 * avg_xgboost[::, 1], color=["#9e9ac8","#74c476","#6baed6","#fc9272"])

ax.set_xticks(x_pos)
ax.set_xticklabels(xgboost_labels)
for x,avg_,sdt_ in zip(x_pos,avg_xgboost[::, 0],avg_xgboost[::, 1]):
    if x==3:
        ax.text(x,0.05,fr"{avg_:.2f}$\pm${sdt_:.2f}",ha='center',rotation=90,fontsize=22)
    else:
        ax.text(x,0.05,fr"{avg_:.2f}",ha='center',rotation=90,fontsize=22)
ax.tick_params(axis="x", rotation=45)
ax.set_ylabel("Acurácia")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y,random_state=np.argmax(acc_xg[::, 1]))

clf_vf = XGBClassifier()
clf_vf.fit(X_train,y_train);
accuracy_score(y_test, clf_vf.predict(X_test))

In [ ]:
f, ax = plt.subplots(1, 1, figsize=stdfigsize(scale=2), sharex=True)

labels = le.inverse_transform(np.arange(min(y), max(y) + 1))

sns.heatmap(
    confusion_matrix(y_test, clf_vf.predict(X_test),normalize='true'),
    ax=ax,
    cmap="magma_r",
    annot=True,
    annot_kws={"fontsize": 25},
    fmt=".2f",
    cbar=False,
)

ax.set_xticklabels(labels, size=20)
ax.set_yticklabels(labels, size=20)
ax.set_xlabel('Previsão do modelo')
ax.set_ylabel('Resposta correta')
ax.tick_params(axis="x", rotation=45)
ax.tick_params(axis="y", rotation=0)

### Grande área

In [ ]:
fig = px.scatter(data,
                 x='umap_bert_ult5_pt_small_embbedding_1',
                 y='umap_bert_ult5_pt_small_embbedding_2',
                 color='grande-area',
                 custom_data=['texto_completo_questao_wrap'],
                 height=1000,
                 width=1200,
)

fig.update_traces(
    hovertemplate =
                "%{customdata[0]}",
    mode='markers',
    marker={'size': 10}
)

fig.show(config = {'scrollZoom': True,
                   })

In [ ]:
grandes_areas = data['grande-area'].unique()
colors = generate_color_list(np.asarray(list(range(len(grandes_areas)))),cmap=palettable.colorbrewer.qualitative.Set1_4.get_mpl_colormap())
markers = ['v', '^', '<', '>', 's', 'p', '*', 'h', 'H', '+', 'x', 'D', 'd', 'P', 'X', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [ ]:
f, ax = plt.subplots(figsize=stdfigsize(scale=2))

for i, area_ in enumerate(grandes_areas):
    ax.scatter(
        data[data['grande-area'] == area_]['umap_bert_ult5_pt_small_embbedding_1'],
        data[data['grande-area'] == area_]['umap_bert_ult5_pt_small_embbedding_2'],
        s=50,
        marker=markers[i],
        c=colors[i],
        label=area_
)

# ax.set_xlim(0,15)
# ax.set_ylim(-2,8)
ax.legend(ncols=1,loc='upper right',bbox_to_anchor=(1.2, 1.0))
ax.set_xlabel('UMAP 1')
ax.set_ylabel('UMAP 2')

In [ ]:
le = LabelEncoder()
y = le.fit_transform(data['grande-area'])
X = np.vstack(data.bert_ult5_pt_small_embbedding.values)

In [ ]:
#fração de questões em cada área
[f"{area_}: {np.mean(data['grande-area']==area_)}" for area_ in data['grande-area'].unique()]

In [ ]:
acc_xg = []
acc_dummy_mf = []
acc_dummy_st = []
acc_dummy_un = []
for i in tqdm.tqdm(range(10)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y,random_state=i)
    
    clf_vf = XGBClassifier()
    
    clf_vf_dummy_mf = DummyClassifier(strategy='most_frequent')
    clf_vf_dummy_st = DummyClassifier(strategy='stratified')
    clf_vf_dummy_un = DummyClassifier(strategy='uniform')
    
    clf_vf.fit(X_train,y_train);
    
    clf_vf_dummy_mf.fit(X_train,y_train);
    clf_vf_dummy_st.fit(X_train,y_train);
    clf_vf_dummy_un.fit(X_train,y_train);
    
    acc_xg_          = accuracy_score(y_train, clf_vf.predict(X_train)),accuracy_score(y_test, clf_vf.predict(X_test))
    acc_dummy_mf_    = accuracy_score(y_train, clf_vf_dummy_mf.predict(X_train)),accuracy_score(y_test, clf_vf_dummy_mf.predict(X_test))
    acc_dummy_st_ = accuracy_score(y_train, clf_vf_dummy_st.predict(X_train)),accuracy_score(y_test, clf_vf_dummy_st.predict(X_test))
    acc_dummy_un_ = accuracy_score(y_train, clf_vf_dummy_un.predict(X_train)),accuracy_score(y_test, clf_vf_dummy_un.predict(X_test))
    acc_xg.append(acc_xg_)
    acc_dummy_mf.append(acc_dummy_mf_)
    acc_dummy_st.append(clf_vf_dummy_st_)
    acc_dummy_un.append(clf_vf_dummy_un_)

In [ ]:
acc_xg = np.asarray(acc_xg)
acc_dummy_mf = np.asarray(acc_dummy_mf)
acc_dummy_st = np.asarray(acc_dummy_st)
acc_dummy_un = np.asarray(acc_dummy_un)

In [ ]:
avg_xgboost = np.asarray(
    [
        [np.mean(acc_dummy_un[::, 1]), np.std(acc_dummy_un[::, 1])],
        [np.mean(acc_dummy_st[::, 1]), np.std(acc_dummy_st[::, 1])],
        [np.mean(acc_dummy_mf[::, 1]), np.std(acc_dummy_mf[::, 1])],
        [np.mean(acc_xg[::, 1]), np.std(acc_xg[::, 1])],
    ]
)

In [ ]:
avg_xgboost[::, 0]

In [ ]:
xgboost_labels = ["Uniforme","Estratificado","Mais frequente","xgboost"]

f, ax = plt.subplots(1, 1, figsize=stdfigsize(), sharex=True)
x_pos = list(range(len(xgboost_labels)))

ax.bar(x_pos, avg_xgboost[::, 0], yerr=1 * avg_xgboost[::, 1], color=["#9e9ac8","#74c476","#6baed6","#fc9272"])

ax.set_xticks(x_pos)
ax.set_xticklabels(xgboost_labels)

for x,avg_,sdt_ in zip(x_pos,avg_xgboost[::, 0],avg_xgboost[::, 1]):
    if x==3:
        ax.text(x,0.05,fr"{avg_:.2f}$\pm${sdt_:.2f}",ha='center',rotation=90,fontsize=22)
    else:
        ax.text(x,0.05,fr"{avg_:.2f}",ha='center',rotation=90,fontsize=22)

ax.tick_params(axis="x", rotation=45)
ax.set_ylabel("Acurácia")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y,random_state=np.argmax(acc_xg[::, 1]))

clf_vf = XGBClassifier()
clf_vf.fit(X_train,y_train);
accuracy_score(y_test, clf_vf.predict(X_test))

In [ ]:
f, ax = plt.subplots(1, 1, figsize=stdfigsize(scale=1), sharex=True)

labels = le.inverse_transform(np.arange(min(y), max(y) + 1))

sns.heatmap(
    confusion_matrix(y_test, clf_vf.predict(X_test),normalize='true'),
    ax=ax,
    cmap="magma_r",
    annot=True,
    annot_kws={"fontsize": 25},
    fmt=".2f",
    cbar=False,
)

ax.set_xticklabels(labels, size=20)
ax.set_yticklabels(labels, size=20)
ax.set_xlabel('Previsão do modelo')
ax.set_ylabel('Resposta correta')
ax.tick_params(axis="x", rotation=45)
ax.tick_params(axis="y", rotation=0)